# Check whether quantum annealing (QA) works properly or not

In [1]:
from functions_SFMA import *

## Load W matrices and mean and standard deviation (std) values

In [2]:
# Parameters
num_N = 6
num_W = 3


In [3]:
n_bit = num_N*num_K # number of spins (qubits)
val_dim = f'd{n_bit}'

# Load W matrices mean and std values
folder0 =  '20241227_pFMA_Wmatrix/results/z900/' 
name0   =  f'C6_{num_N}x2'  
init_file = f'{folder0}/{name0}.pickle.bz2'
with bz2.open(init_file, 'rb') as f:
        param = pickle.load(f)

# Load the target W matrix 
list_Wmatrix = [] # list for collecting the W matrices
for w in param['tdata']:
     list_Wmatrix.append(w) # append the W matrix to list_Wmatrix
val_num_W = f'W_{num_W}'
Matrix_W = list_Wmatrix[num_W] # target W matrix

# Load the mean and std values
obj_mean = param['obj_func_mean'] # array of the mean values of the objective functions (output values)
obj_std = param['obj_func_std']   # array of the standard deviation (std) values of the outputs
mean = obj_mean[num_W] # mean value for the index num_W
std = obj_std[num_W] # std value for the index num_W

## Introduce an inital dataset, train a standardized Factorization Machine (FM) model using it, and search the optimal solution via QA. Further, perform an exhaustive (brute force) search method.   

In [4]:
# Parameters for an initial dataset and a FM model
n_in = n_bit # size of an initial dataset
k_FM = int(n_bit/2)-1 # hyperparameter for a FM model 

# Generate an inital dataset.
xin, yin = init_training_data(Matrix_W, num_N, num_K, n_bit, n_in, nin_seed) # inital dataset
yin_standard = standard(yin, mean, std) # standardized output value


# Train a standardized FM model.
torch.manual_seed(seed_intial_FMTorch) # set a random seed for reproducibility 
model_FM_standard = TorchFM(d=n_bit, k=k_FM, Stand_Dev=std) # instantiate a FM model 
trained_FMmodel_standard = train_FM_model(xin, yin_standard, model_FM_standard) # train a FM model
_, trained_parameters_standard = trained_FMmodel_standard # get FM model parameters (v,w,w0) 
v_standard, w_standard, w0_standard = trained_parameters_standard # get v,w, and w0 
M_QUBO_standard = construct_QUBO(w_standard, v_standard) # construct a QUBO matrix using w and v

all_binaries = all_binaries(n_bit) # all binary configurations
list_QUBO_standard = [] # list of the values of a FM function for all binaries
for x in all_binaries:
    y = calculate_binary_quadratic(x, M_QUBO_standard, w0_standard) # value of the FM function
    list_QUBO_standard.append(y) # append y to list_QUBO_standard 
array_QUBO_standard = np.array(list_QUBO_standard) # convert list_QUBO_standard to a NumPy array

# exhaustive search
sorted_binary_indices_QUBO_standard =  np.argsort(array_QUBO_standard) # sorted indices of the output values
xstar_exhaustive_decimal = sorted_binary_indices_QUBO_standard[0] # index of the optimal solution in a binary solution space

# QA
xstar_standard, ystar_standard = quantum_anneal(trained_FMmodel_standard, Matrix_W, num_N, num_K) 
# xstar_standard: best candiate solution
# ystar_standard: output value of the best candiate solution: the minimum value
xstar_QA_decimal = binary_array_to_decimal(xstar_standard) # convert a binary vector into a decimal: the index of the best candidate binary solution in a binary solution space

## Compare the best candidate (optimal) solution obtained by QA with that achieved by the exhaustive search methods

In [5]:
print(val_num_W, "QA:", xstar_QA_decimal, "exaustive:", xstar_exhaustive_decimal)

W_3 QA: 1484 exaustive: 1484
